In [1]:
import pickle
import numpy as np
import pandas as pd
from pathlib import Path

%matplotlib inline

In [2]:
load_dir = Path('../2_pipeline').resolve()
model_name = '13_Final_Model_Training'

nw_dir = load_dir.joinpath('10_Semmantic_Compression', 'out')
hyp_dir = load_dir.joinpath('11c_Model_Prep_Hyperparam_tuning', 'out')
model_dir = load_dir.joinpath(model_name, 'out')

In [3]:
nodes = pd.read_csv(nw_dir.joinpath('nodes_biolink.csv'), dtype=str)
edges = pd.read_csv(nw_dir.joinpath('edges_biolink.csv'), dtype=str)

In [4]:
best_param = pickle.load(open(hyp_dir.joinpath('best_param.pkl'), 'rb'))

In [5]:
from hetnet_ml.extractor import MatrixFormattedGraph
mg = MatrixFormattedGraph(nodes, edges, 'ChemicalSubstance', 'Disease', max_length=4, w=best_param['w'], n_jobs=30)

Processing node and edge data...
Initializing metagraph...
Generating adjacency matrices...


100%|██████████| 96/96 [02:01<00:00,  1.27s/it]



Determining degrees for each node and metaedge


100%|██████████| 96/96 [00:32<00:00,  2.92it/s]



Weighting matrices by degree with dampening factor 0.5267957740707717...


100%|██████████| 96/96 [00:02<00:00, 46.30it/s]


In [6]:
model = pickle.load(open(model_dir.joinpath('model.pkl'), 'rb'))

/opt/mmayers/software/anaconda3/envs/ml/lib/python3.6/site-packages/data_tools/df_processing.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
metapaths = pd.read_csv(hyp_dir.joinpath('kept_features.txt'), header=None)[0].tolist()

In [8]:
from data_tools.graphs import dataframes_to_metagraph, metapaths_to_json
metagraph = dataframes_to_metagraph(nodes, edges)

all_metapaths = metagraph.extract_metapaths('ChemicalSubstance', 'Disease', 4)
mp_info = {mp.abbrev: mp for mp in all_metapaths if mp.abbrev in metapaths}

In [9]:
res = pd.read_csv(model_dir.joinpath('results.csv'))
coef = pd.read_csv(model_dir.joinpath('coef.csv'))

In [10]:
res.head(2)

,chemicalsubstance_id,disease_id,status,proba
0,CHEBI:100,DOID:0002116,0,0.006045
1,CHEBI:100,DOID:0014667,0,0.005136


In [11]:
len(res)

85126262

In [12]:
id_to_name = nodes.set_index('id')['name'].to_dict()
res['chem_name'] = res['chemicalsubstance_id'].map(id_to_name)
res['dis_name'] = res['disease_id'].map(id_to_name)

In [13]:
def find_nodes_by_name(sub_str, n_type=None):
    if n_type is not None:
        qr = nodes.query('label == @n_type')   
    else:
        qr = nodes
    
    return qr[qr['name'].str.contains(sub_str)]

In [14]:
nodes['name'] = nodes['name'].fillna('')

In [15]:
find_nodes_by_name('lzheimer', n_type='Disease')

,id,name,label,xrefs,source
33912,DOID:0080348,Alzheimer's disease 1,Disease,DOID:0080348|OMIM:104300,NaN
34225,DOID:0110036,Alzheimer disease 16,Disease,DOID:0110036|MONDO:0010422|OMIM:300756|UMLS:C2...,NaN
34226,DOID:0110037,Alzheimer disease 5,Disease,DOID:0110037|MESH:C566578|MONDO:0011194|OMIM:6...,NaN
34227,DOID:0110038,Alzheimer disease 6,Disease,DOID:0110038|MESH:C565325|MONDO:0011561|OMIM:6...,NaN
34228,DOID:0110039,Alzheimer disease 7,Disease,DOID:0110039|MESH:C565251|MONDO:0011647|OMIM:6...,NaN
34229,DOID:0110041,Alzheimer disease 8,Disease,DOID:0110041|MESH:C564622|MONDO:0011777|OMIM:6...,NaN
34230,DOID:0110042,Alzheimer disease 3,Disease,DOID:0110042|MONDO:0011913|OMIM:607822|UMLS:C1...,NaN
34231,DOID:0110043,Alzheimer disease 10,Disease,DOID:0110043|MESH:C566465|MONDO:0012321|OMIM:6...,NaN
34232,DOID:0110044,Alzheimer disease 11,Disease,DOID:0110044|MESH:C565228|MONDO:0012344|OMIM:6...,NaN
34233,DOID:0110045,Alzheimer disease 12,Disease,DOID:0110045|MESH:C567022|MONDO:0012609|OMIM:6...,NaN


In [16]:
alz_id = 'DOID:10652'

In [17]:
alz_res = res.query('disease_id == @alz_id')
len(alz_res)

12449

In [18]:
alz_res.sort_values('proba', ascending=False).head(10)

,chemicalsubstance_id,disease_id,status,proba,chem_name,dis_name
20262326,CHEBI:42944,DOID:10652,1,0.969681,galanthamine,Alzheimer's disease
10046354,CHEBI:2666,DOID:10652,0,0.856627,amitriptyline,Alzheimer's disease
84724152,UNII:OHF2F5C0ZH,DOID:10652,0,0.851006,Tetrachlorodibenzodioxin,Alzheimer's disease
80737598,MESH:D002800,DOID:10652,1,0.849316,Cholinesterase Inhibitors,Alzheimer's disease
12056726,CHEBI:29678,DOID:10652,0,0.849038,sodium arsenite,Alzheimer's disease
7413724,CHEBI:16240,DOID:10652,0,0.843131,hydrogen peroxide,Alzheimer's disease
19475956,CHEBI:39162,DOID:10652,0,0.841937,(R)-nicotine,Alzheimer's disease
22019692,CHEBI:48883,DOID:10652,0,0.830595,physostigmine salicylate,Alzheimer's disease
11803720,CHEBI:29081,DOID:10652,1,0.822403,synephrine,Alzheimer's disease
82679590,MESH:D014028,DOID:10652,0,0.814261,Tobacco Smoke Pollution,Alzheimer's disease


In [21]:
pos_coef = coef.query('coef > 0')['feature'].tolist()

In [22]:
msat = model[0]
max_abs = model[1]

In [23]:
ini_means = {f: m for f, m in zip(metapaths, msat.initial_mean_)}
ma_scale = {f: m for f, m in zip(metapaths, max_abs.scale_)}
feat_coef = coef.set_index('feature')['coef'].to_dict()
feat_zcoef = coef.set_index('feature')['zcoef'].to_dict()

In [24]:
def get_model_metric(path_df):
    totals = path_df.groupby('metapath')['metric'].sum().to_dict()
    percent_of_total = path_df['metric'] / path_df['metapath'].map(totals)
    
    trans_metric = np.arcsinh(path_df['metapath'].map(totals) / path_df['metapath'].map(ini_means))
    scal_metric = trans_metric / path_df['metapath'].map(ma_scale)
    
    model_metric = scal_metric * path_df['metapath'].map(feat_coef) * percent_of_total
    return model_metric

In [25]:
def get_std_model_metric(path_df):
    
    totals = path_df.groupby('metapath')['metric'].sum().to_dict()
    percent_of_total = path_df['metric'] / path_df['metapath'].map(totals)
    
    trans_metric = np.arcsinh(path_df['metric'] / path_df['metapath'].map(ini_means))
    scal_metric = trans_metric / path_df['metapath'].map(ma_scale)
    
    model_metric = scal_metric * path_df['metapath'].map(feat_zcoef) * percent_of_total
    return model_metric

In [26]:
def get_path_info(compound, disease, n_jobs=30):
    path_df = pd.DataFrame(mg.extract_paths(compound, disease, pos_coef, n_jobs=n_jobs))
    path_df['model_metric'] = get_model_metric(path_df)
    path_df['std_metric'] = get_std_model_metric(path_df)
    
    return path_df

In [27]:
def draw_top_paths(path_df, head_num=10, metric='model_metric', poi_idx=None, proba=None):
        
    top_n_paths = path_df.sort_values(metric, ascending=False).head(head_num)

    # Get the list of paths
    list_of_paths = top_n_paths['node_ids'].tolist()
    path_weights = top_n_paths[metric].tolist()

    # Get the edge names
    list_of_edges = []
    for mp in top_n_paths['metapath']:
        list_of_edges.append(gp.parse_metapath_to_edge_names(mp, mp_info, inv_map))


    
    if type(poi_idx) == list:
        ec = []
        for idx in poi_idx:
            poi = path_df.loc[idx, 'node_ids']
            ec.append(gp.highlight_path_of_interest(list_of_paths, poi))
        
        # Take the edges from the first highlighted poi
        edge_color_map = ec[0]
        for ecm in ec[1:]:
            for k, v in ecm.items():
                # Any other highlihted paths get marked
                # seaborn 1 is highlight color.
                if v == sns.color_palette().as_hex()[1]:
                    edge_color_map[k] = v
        
    elif poi_idx is not None:
        path_of_interest = path_df.loc[poi_idx, 'node_ids']
        edge_color_map = gp.highlight_path_of_interest(list_of_paths, path_of_interest)
    else:
        path_of_interest = []
        edge_color_map = gp.highlight_path_of_interest(list_of_paths, path_of_interest)
    
    

    G = gp.build_explanitory_graph(list_of_paths, list_of_edges, path_weights=path_weights, 
                                node_id_to_label=id_to_label, node_id_to_color=node_id_to_color, 
                                edge_id_to_color=edge_color_map, min_dist=3)

    gp.draw_explanitory_graph(G, node_id_to_name=id_to_name, node_names=None, proba=proba, n_paths=10, xscale=17);

In [28]:
# Choose custom node colors
node_color_map = {'AnatomicalEntity': '#5252b9',
                'BiologicalProcessOrActivity': '#fc99d5',
                'MacromolecularMachine': '#d032a3',
                'Disease': '#50e316',
                'Pathway': '#b7d165',
                'PhenotypicFeature': '#165f28',
                'GeneFamily': '#6404b7',
                'PhysiologicalProcess': '#ec102f',
                'OrganismTaxon': '#e9bf98',
                'ChemicalSubstance': '#21f0b6'}

node_id_to_color = nodes.set_index('id')['label'].map(node_color_map).to_dict()
id_to_label = nodes.set_index('id')['label'].to_dict()

edge_info = pd.read_csv('../0_data/manual/edge_semtypes.csv')
inv_map = edge_info.set_index('fwd_edge')['rev_edge'].to_dict()

import graph_plots as gp


In [29]:
import warnings
warnings.simplefilter('ignore')

In [27]:
ref_c = nodes[~nodes['source'].isnull()]['id'].unique()
len(ref_c)

5988

### Get probabilites of treatment for ReFrame to Alzheimers

In [28]:
(ref_pairs, out_mp), dwpc = mg.extract_dwpc(metapaths=metapaths, start_nodes=ref_c, 
                                        end_nodes=[alz_id], n_jobs=30, return_sparse=True, sparse_df=False)

Preparing function arguments...
Calculating DWPCs...


100%|██████████| 196/196 [03:18<00:00,  1.01s/it]



Reshaping Result Matrices...


100%|██████████| 196/196 [00:00<00:00, 943.73it/s]

Stacking columns...


In [29]:
ref_pairs['proba'] = model.predict_proba(dwpc)[:, 1]

In [30]:
ref_pairs['chem_name'] = ref_pairs['chemicalsubstance_id'].map(id_to_name)
ref_pairs['dis_name'] = ref_pairs['disease_id'].map(id_to_name)

### Define plotting function

In [36]:
# Choose custom node colors
node_color_map = {'AnatomicalEntity': '#5252b9',
                'BiologicalProcessOrActivity': '#fc99d5',
                'MacromolecularMachine': '#d032a3',
                'Disease': '#50e316',
                'Pathway': '#b7d165',
                'PhenotypicFeature': '#165f28',
                'GeneFamily': '#6404b7',
                'PhysiologicalProcess': '#ec102f',
                'OrganismTaxon': '#e9bf98',
                'ChemicalSubstance': '#21f0b6'}

node_id_to_color = nodes.set_index('id')['label'].map(node_color_map).to_dict()
id_to_label = nodes.set_index('id')['label'].to_dict()

edge_info = pd.read_csv('../0_data/manual/edge_semtypes.csv')
inv_map = edge_info.set_index('fwd_edge')['rev_edge'].to_dict()

import graph_plots as gp

def draw_top_paths(path_df, head_num=10, metric='model_metric', poi_idx=None, proba=None):


    if poi_idx is not None:
        path_of_interest = path_df.loc[poi_idx, 'node_ids']
    else:
        path_of_interest = []
        
    top_n_paths = path_df.sort_values(metric, ascending=False).head(head_num)

    # Get the list of paths
    list_of_paths = top_n_paths['node_ids'].tolist()
    path_weights = top_n_paths[metric].tolist()

    # Get the edge names
    list_of_edges = []
    for mp in top_n_paths['metapath']:
        list_of_edges.append(gp.parse_metapath_to_edge_names(mp, mp_info, inv_map))

    edge_color_map = gp.highlight_path_of_interest(list_of_paths, path_of_interest)

    G = gp.build_explanitory_graph(list_of_paths, list_of_edges, path_weights=path_weights, 
                                node_id_to_label=id_to_label, node_id_to_color=node_id_to_color, 
                                edge_id_to_color=edge_color_map)

    gp.draw_explanitory_graph(G, node_id_to_name=id_to_name, proba=proba, n_paths=len(list_of_paths));


In [37]:
# networkx throws LOTS of matplotlib warnings when plotting, so we're silencing
import warnings
warnings.simplefilter('ignore')

## Save out results

In [43]:
from tqdm import tqdm

this_name = '14e_Model_results_analysis_alzheimers'
out_dir = load_dir.joinpath(this_name, 'out', 'ref_alz')

out_dir.mkdir(exist_ok=True, parents=True)

In [56]:
treats = set(edges.query('abbrev == "CtD"')[['start_id', 'end_id']].apply(tuple, axis=1))

In [58]:
status = np.array([{(row.chemicalsubstance_id, row.disease_id)} & treats for row in ref_pairs.itertuples()]).astype(bool).astype(int)

In [62]:
ref_pairs['status'] = status

In [64]:
(ref_pairs.sort_values('proba', ascending=False)
          .head(100)
          .to_csv(out_dir.parent.joinpath('alz_extract_path_results.csv'), index=False))

### Get paths for each compound in top 100 and save 100 paths

In [45]:
finished = 0
to_go = 100 - finished

skipped = []

for line in tqdm(ref_pairs.sort_values('proba', ascending=False).head(100).tail(to_go).itertuples(), total=to_go):
    
    try:
        this_res = get_path_info(line.chemicalsubstance_id, line.disease_id)
    except:
        this_res = get_path_info(line.chemicalsubstance_id, line.disease_id, 1)
        
    out_name = line.chemicalsubstance_id.replace(':', '_') + '_' + line.disease_id.replace(':', '_')
    out_name += '_{}.csv'.format(line.proba)

    print(out_dir.joinpath(out_name))
    this_res.sort_values('model_metric', ascending=False).head(100).to_csv(out_dir.joinpath(out_name), index=False)


  1%|          | 1/100 [00:31<52:08, 31.60s/it]

/opt/mmayers/projects/metapaths/2_pipeline/14e_Model_results_analysis_alzheimers/out/ref_alz/CHEBI_42944_DOID_10652_0.9696804881095886.csv



  2%|▏         | 2/100 [00:58<49:05, 30.06s/it]

/opt/mmayers/projects/metapaths/2_pipeline/14e_Model_results_analysis_alzheimers/out/ref_alz/CHEBI_41934_DOID_10652_0.8537576794624329.csv



100%|██████████| 113/113 [05:36<00:00,  2.97s/it]


...

100%|██████████| 113/113 [00:45<00:00,  2.47it/s]


/opt/mmayers/projects/metapaths/2_pipeline/14e_Model_results_analysis_alzheimers/out/ref_alz/CHEBI_31530_DOID_10652_0.4750608801841736.csv


100%|██████████| 113/113 [03:08<00:00,  1.67s/it]


/opt/mmayers/projects/metapaths/2_pipeline/14e_Model_results_analysis_alzheimers/out/ref_alz/CHEBI_16602_DOID_10652_0.47435009479522705.csv


100%|██████████| 100/100 [18:17:25<00:00, 658.45s/it]


### Save images of top 10 paths for each of the top 100 compounds

In [46]:
from data_tools.df_processing import strip_special_chars

In [47]:
import warnings
warnings.simplefilter('ignore')

In [49]:
import matplotlib.pyplot as plt

In [50]:
img = out_dir.joinpath('img')
img.mkdir(exist_ok=True, parents=True)

for f in out_dir.iterdir():
    if f.name.endswith('.csv'):

        prob = float(f.name.split('_')[-1].rstrip('.csv'))
        path_info = pd.read_csv(f, converters={'nodes': eval, 'node_ids': eval}).head(10)
        d_name = path_info.loc[0, 'nodes'][0]
        d_name = strip_special_chars(d_name.lower().replace(' ', '_'))
        
        draw_top_paths(path_info, 10, proba=prob)

        plt.savefig(img.joinpath('{}_{:1.3f}.png'.format(d_name, prob)), dpi=300)
        plt.clf()

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

...

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>